- references
    - https://github.com/DeepRLChinese/DeepRL-Chinese/blob/master/09_trpo.py

In [11]:
import torch
from torch import nn
from collections import Counter
from torch.distributions.categorical import Categorical

In [36]:
# 类别型概率分布的采样
c = Counter()
m = Categorical(probs=torch.tensor([0.25, 0.25, 0.25, 0.25]))
for _ in range(500):
    c.update([m.sample().numpy().tolist()])
print(c)
print(Counter(m.sample((500, )).numpy().tolist()))

Counter({0: 143, 2: 122, 1: 119, 3: 116})
Counter({1: 138, 0: 137, 3: 113, 2: 112})


In [2]:
class CategoricalLayer(nn.Module):
    """
    Implements a layer that outputs a multinomial distribution
    Methods
    ------
    __call__(log_action_probs)
        Takes as input log probabilities and outputs a pytorch multinomail
        distribution
    """

    def __init__(self):
        super().__init__()

    def __call__(self, log_action_probs):
        return Categorical(logits=log_action_probs)

In [ ]:
class PolicyNet(nn.Module):
    def __init__(self, dim_obs, num_act):
        super().__init__()
        self.fc1 = nn.Linear(dim_obs, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, num_act)
        self.log_softmax = nn.LogSoftmax(dim=-1)
        self.categorical = CategoricalLayer()

    def forward(self, obs):
        x = F.relu(self.fc1(obs))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)  # logits
        x = self.log_softmax(x)
        x = self.categorical(x)
        return x